In [1]:
#importing libraries
import pandas as pd
import nltk
import re
import numpy as np
from pandas import StringDtype
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [2]:
#importing the dataset
train=pd.read_csv('train.csv')
Xtrain=train.copy()



In [3]:
#selecting the columns for train input and labels
X1=Xtrain.iloc[:,2]
Y1=Xtrain.iloc[:,[1]]

X1.astype("string")

0         @user when a father is dysfunctional and is s...
1        @user @user thanks for #lyft credit i can't us...
2                                      bihday your majesty
3        #model   i love u take with u all the time in ...
4                   factsguide: society now    #motivation
                               ...                        
31957    ate @user isz that youuu?ðððððð...
31958      to see nina turner on the airwaves trying to...
31959    listening to sad songs on a monday morning otw...
31960    @user #sikh #temple vandalised in in #calgary,...
31961                     thank you @user for you follow  
Name: tweet, Length: 31962, dtype: string

In [4]:
for i in range (train.shape[0]):
    X1[i]=re.sub(r"@[A-Za-z0-9_]+"," ",X1[i])
    X1[i]=re.sub(r"[^a-zA-Z0-9]+"," ",X1[i])
    X1[i]=re.sub(r'^https?:\/\/.*[\r\n]*', ' ',X1[i] , flags=re.MULTILINE)



<ipython-input-4-755ea6698574>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1[i]=re.sub(r"@[A-Za-z0-9_]+"," ",X1[i])
<ipython-input-4-755ea6698574>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1[i]=re.sub(r"[^a-zA-Z0-9]+"," ",X1[i])
<ipython-input-4-755ea6698574>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1[i]=re.sub(r'^https?:\/\/.*[\r\n]*', ' ',X1[i] , flags=re.MULTILINE)


In [5]:
nltk.download('punkt')

#tokenizer
X1=Xtrain.iloc[:,2].apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pranj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
#remoing stop words 
en_stops = set(stopwords.words('english'))
en_stops.remove('not')
en_stops.remove('nor')
en_stops.remove('no')


  
#stemming
ps = PorterStemmer()
for j in range(train.shape[0]):
    st=[]
    st=[word for word in X1[j] if word.lower() not in en_stops]
    ft=[]
    for w in st:
     ft.append(ps.stem(w))
    X1[j]=ft

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size = 0.05)


X_train=[" ".join(review) for review in X_train]
X_test=[" ".join(review) for review in X_test]


#vectorizer
vectorizer = TfidfVectorizer(max_features=20000)

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [8]:
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train.values.ravel())


y_pred = classifier.predict(X_test)

In [9]:


cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[1504    3]
 [  43   49]]
0.9712320200125079
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      1507
           1       0.94      0.53      0.68        92

    accuracy                           0.97      1599
   macro avg       0.96      0.77      0.83      1599
weighted avg       0.97      0.97      0.97      1599

